In [1]:
using Zygote
using Flux: σ, softmax, crossentropy, Chain, Optimise, onehotbatch, onehot, onecold
using Flux.Data: MNIST
using SparseArrays
using StatsBase: sample, shuffle, mean
#using Plots

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /home/olszewskip/.julia/packages/CUDAdrv/mCr0O/src/CUDAdrv.jl:69


In [79]:
labels = MNIST.labels();
label_pool = sort(Vector(collect(Set(labels))));
y = onehotbatch(labels, label_pool);

In [3]:
imgs = MNIST.images();

# lighter_than(pixel, threshold) = pixel.val > threshold

# function lightness_threshold(pixel_array::Array{T, 2}, threshold) where T
#     convert.(T, lighter_than.(pixel_array, threshold))
# end

# imgs_blackenwhite = lightness_threshold.(imgs, 0.7);
#X = hcat(convert.(Array{Float32, 1}, reshape.(imgs_blackenwhite, :))...);
X = hcat(convert.(Array{Float32, 1}, reshape.(imgs, :))...);

In [4]:
imgs[10_000]

In [5]:
# function spones(m, n, fraq)
#     nz_count = ceil(Integer, m * n * fraq)
#     return sparse(sample(1:m, nz_count), sample(1:n, nz_count), ones(nz_count), m, n)
# end

In [63]:
struct SprAffine{S,T,F}
    W::S
    b::T
    #frac::U
    σ::F
end

function Base.show(io::IO, layer::SprAffine)
  print(io, "SprAffine(", size(layer.W, 2), ", ", size(layer.W, 2))
  print(io, ", ", round(length(layer.W.nzval) / length(layer.W), sigdigits=6))
  layer.σ == identity || print(io, ", ", layer.σ)
  print(io, ")")
end

SprAffine(in::Number, out::Number, frac::AbstractFloat, σ::Function=identity) =
    #SprAffine(spones(out, in, frac), spzeros(out), fraq, σ)
    SprAffine(sprandn(Float32, out, in, frac), zeros(Float32, out), σ)

(layer::SprAffine)(x) = layer.σ.(layer.W * x .+ layer.b)

In [11]:
loss(model, x, y) = crossentropy(Array(softmax(model(x))), y)

loss (generic function with 1 method)

In [12]:
accuracy(model, X, y) = mean(onecold(model(X)) .== onecold(y))

accuracy (generic function with 1 method)

In [27]:
# function Optimise.apply!(opt::Optimise.Descent, arr::AbstractSparseArray, d_arr::AbstractSparseArray)
#     val_ptr = 1
#     col_val = 1
#     for (rows_begin, rows_end) in (zip(arr.colptr[1:end-1], arr.colptr[2:end]))
#         for row_ptr in rows_begin:(rows_end - 1)
#             row_val = arr.rowval[row_ptr]
#             arr.nzval[val_ptr] -= d_arr[row_val, col_val] * opt.eta
#             val_ptr += 1
#         end
#         col_val += 1
#     end
# end

In [37]:
update_!(opt, model, grads::Nothing) = model

function update_!(opt, arr::AbstractSparseMatrix, d_arr::AbstractSparseMatrix)
    dropzeros!(d_arr)
    val_ptr = 1
    col_val = 1
    for (rows_begin, rows_end) in (zip(arr.colptr[1:end-1], arr.colptr[2:end]))
        for row_ptr in rows_begin:(rows_end - 1)
            row_val = arr.rowval[row_ptr]
            arr.nzval[val_ptr] -= d_arr[row_val, col_val] * opt.eta
            val_ptr += 1
        end
        col_val += 1
    end
    return arr
end

function update_!(opt, arr::AbstractArray, d_arr::AbstractArray)
    Optimise.apply!(opt, arr, d_arr)
    arr .-= d_arr
    return arr
end

function update_!(opt, model, grads)
    @assert nfields(model) == nfields(grads)
            ["nfields(model) $(nfields(model)) ≠ nfields(grads) $(nfields(grads))"]
#     if nfields(model) == 0
#         return model
#     end
    
#     println(level)
#     println(model)
#     println(typeof(model))        
#     println(grads)
#     println(typeof(grads))
#     println(" --- ")
    
    for field_idx in 1:nfields(model)
        field = getfield(model, field_idx)
        d_field = getfield(grads, field_idx)
        update_!(opt, field, d_field)
    end

#     println("ha")
    return model
end

update_! (generic function with 4 methods)

In [64]:
model =
Chain(
    SprAffine(size(X, 1), 100, 0.1),
    SprAffine(100, size(y, 1), 0.1),
    softmax
);

In [76]:
softmax(Array(model(sparse(X[:, 1]))))

10-element Array{Float32,1}:
 0.09788819 
 0.20432639 
 0.08693092 
 0.08693092 
 0.08693236 
 0.08862153 
 0.08693093 
 0.08757196 
 0.0869358  
 0.086931065

In [74]:
gradient(model) do model
    crossentropy(softmax(Array(model(sparse(X[:, 1])))), y[:, 1])
end

ErrorException: Need an adjoint for constructor SparseVector{Float32,Int64}. Gradient is of type SparseVector{Float32,Int64}

In [12]:
# prune!(x) = (x) -> nothing

# function prune!(layer::SprAffine)
#     new_length = ceil(Int, layer.fraq * length(layer.W))
#     if length(layer.W.nzval) > new_length
#         I, J, V = findnz(layer.W)
#         indices = sortperm(abs.(V), rev=true)[1:new_length]
#         layer.W[:,:] = sparse(I[indices], J[indices], V[indices], size(layer.W)...)
#         dropzeros!(layer.W)
#     end
#     return nothing
# end

augment!(x, y) = nothing

function augment!(layer::SprAffine, frac::AbstractFloat)
    # redraw fraction of the layer's Weights
    I, J, V = findnz(layer.W)
    len_augmented = floor(Int, frac * length(V))
    len_preserved = length(V) - len_augmented
    elems = Set(zip(I, J))
    indices = sortperm(abs.(V), rev=true)[1:len_preserved]
    I = I[indices]
    J = J[indices]
    V = V[indices]
    I_ = similar(I, len_augmented)
    J_ = similar(J, len_augmented)
    V_ = similar(V, len_augmented)
    index = 1
    while index <= len_augmented
        i = sample(1:size(layer.W, 1))
        j = sample(1:size(layer.W, 2))
        if (i,j) in elems
            continue
        end
        push!(elems, (i, j))
        I_[index] = i
        J_[index] = j
        V_[index] = randn()
        index += 1
    end
    append!(I, I_)
    append!(J, J_)
    append!(V, V_)
    layer.W[:, :] = sparse(I, J, V)
    #dropzeros!(layer.W)
end

augment! (generic function with 2 methods)

In [19]:
perm = shuffle(1:size(X,2));
X_ = sparse(X[:, perm]);
y_ = sparse(y[:, perm]);

In [56]:
loss_local, back = pullback(model) do model
    loss(model,
         X_[:, 1:64],
         y_[:, 1:64])
end;
grads = back(1)[1];

In [27]:
length(model[1].W.nzval)

7860

In [28]:
mean(abs.(model[1].W.nzval))

0.7976474738477377

In [29]:
length(getfield(getfield(grads[1], 1), 1).nzval)

32495

In [60]:
getfield(getfield(getfield(grads, 1), 1), 2)

100-element Array{Float32,1}:
  0.0          
  6.239787e-5  
 -8.8716755e-5 
  0.0          
 -9.274957e-7  
  4.150278e-5  
  0.00042373422
 -0.00018793644
  0.0          
  0.00064899493
  5.079318e-5  
 -0.00041047498
 -7.8625424e-5 
  ⋮            
  0.0          
  7.127426e-5  
 -3.0796429e-7 
  1.6695922e-5 
 -1.7569853e-5 
 -9.143843e-5  
 -7.312006e-5  
 -5.7926272e-6 
  0.00010423434
 -0.00043674675
  2.4263459e-6 
  2.7575272e-5 

In [58]:
opt = Optimise.Descent();
update_!(opt, model, grads)

Chain(SprAffine(784, 784, 0.100306), SprAffine(100, 100, 0.096), softmax)

In [26]:
I_, J_, V_ = findnz(model[1].W)
elems_ = Dict(zip(zip(I_, J_), V_))

Dict{Tuple{Int64,Int64},Float64} with 789 entries:
  (10, 782) => 0.0364372
  (2, 435)  => -0.876809
  (4, 366)  => -0.441326
  (6, 141)  => 0.611518
  (8, 207)  => -0.0166006
  (5, 292)  => 1.20332
  (9, 34)   => 0.855381
  (3, 392)  => -0.51363
  (8, 325)  => 0.21956
  (1, 698)  => 0.493947
  (7, 681)  => 0.220955
  (3, 111)  => -0.250724
  (6, 41)   => 0.762957
  (8, 177)  => 1.29929
  (7, 19)   => -1.35991
  (5, 706)  => -1.70867
  (2, 721)  => -0.81711
  (2, 217)  => 0.132668
  (4, 86)   => 2.23104
  (9, 693)  => 1.06458
  (8, 163)  => 0.245222
  (5, 302)  => 0.398357
  (6, 698)  => -0.211706
  (10, 289) => -1.89186
  (9, 366)  => -0.155154
  ⋮         => ⋮

In [138]:
elems_[(10, 441)]

0.001969933869690529

In [139]:
elems[(10, 441)]

KeyError: KeyError: key (10, 441) not found

In [140]:
model[1].W[10, 441]

0.001969933869690529

In [137]:
setdiff(keys(elems_), keys(elems))

Set(Tuple{Int64,Int64}[(10, 441), (10, 207), (7, 317), (2, 435), (6, 158), (1, 151), (2, 654), (5, 604), (10, 411), (8, 207)  …  (2, 232), (7, 291), (2, 565), (3, 341), (9, 684), (6, 362), (9, 480), (4, 298), (8, 606), (3, 345)])

In [189]:
length(model[1].W.nzval)

7870

In [108]:
prune!(model[1]);

In [109]:
length(model[1].W.nzval)

7840

In [87]:
function train_augmenting!(loss, model, data, batch_size, opt)
    I, J, _ = findnz(model[1].W)
    elems = Set(zip(I, J))
    X, y = data
    perm = shuffle(1:size(X,2))
    X_ = sparse(X[:, perm])
    y_ = sparse(y[:, perm])
    batch_indices = 1:batch_size:(size(X,2) - batch_size)
    for (index, batch_index) in enumerate(batch_indices)
        println("Batch $index/$(length(batch_indices))")
        loss_local, back = pullback(model) do model
            loss(model,
                 X_[:, 1:64],
                 y_[:, 1:64])
        end;
        println("Loss: $loss_local")
        grads = back(1)[1];
        update_!(opt, model, grads)
        I, J, _ = findnz(model[1].W)
        elems_ = Set(zip(I, J))
        update_count = length(setdiff(elems_, elems))
        println("$update_count new elements in the first layer")
        elems = elems_
        for layer in model.layers
            augment!(layer, 0.05)
        end
    end
end

train_augmenting! (generic function with 1 method)

In [90]:
train_augmenting!(loss, model, (X[:, 1:1280], y[:, 1:1280]), 128, Optimise.Descent(0.1))

Batch 1/9
Loss: 2.3007128281989955
0 new elements in the first layer
Batch 2/9
Loss: 2.3286548046606255
379 new elements in the first layer
Batch 3/9
Loss: 2.349580478485363
379 new elements in the first layer
Batch 4/9
Loss: 2.3650123226166646
379 new elements in the first layer
Batch 5/9
Loss: 2.375879490606386
379 new elements in the first layer
Batch 6/9
Loss: 2.39537636402858
379 new elements in the first layer
Batch 7/9
Loss: 2.41174763076019
379 new elements in the first layer
Batch 8/9
Loss: 2.3945854864963065
379 new elements in the first layer
Batch 9/9
Loss: 2.405240227713101
379 new elements in the first layer


In [236]:
accuracy(model, X, y)

0.09598333333333334

In [112]:
perm = shuffle(1:size(X,2))
X_ = sparse(X[:, perm])
y_ = sparse(y[:, perm])

batch_size = 64
batch_indices = 1:batch_size:(size(X,2) - batch_size)

opt = Optimise.Descent();

for (index, batch_index) in enumerate(batch_indices)
    println("Batch $index/$(length(batch_indices))")
    grads = gradient(model) do model
        loss(model,
             X_[:, batch_index:(batch_index + batch_size)],
             y_[:, batch_index:(batch_index + batch_size)])
    end[1]
    update_!(opt, model, grads)
    for layer in model.layers
        prune!(layer)
    end
end

Batch 1/937
Batch 2/937
Batch 3/937
Batch 4/937
Batch 5/937
Batch 6/937
Batch 7/937
Batch 8/937
Batch 9/937
Batch 10/937
Batch 11/937
Batch 12/937
Batch 13/937
Batch 14/937
Batch 15/937
Batch 16/937
Batch 17/937
Batch 18/937
Batch 19/937
Batch 20/937
Batch 21/937
Batch 22/937
Batch 23/937


InterruptException: InterruptException:

In [119]:
y_pred = model(sparse(X))

10×60000 SparseMatrixCSC{Float64,Int64} with 600000 stored entries:
  [1 ,     1]  =  3.46267e-25
  [2 ,     1]  =  1.0
  [3 ,     1]  =  2.84968e-27
  [4 ,     1]  =  2.77862e-29
  [5 ,     1]  =  2.51108e-38
  [6 ,     1]  =  1.90558e-48
  [7 ,     1]  =  1.51876e-49
  [8 ,     1]  =  1.45816e-34
  [9 ,     1]  =  4.13299e-38
  [10,     1]  =  2.7363e-25
  [1 ,     2]  =  1.76238e-31
  [2 ,     2]  =  1.0
  ⋮
  [8 , 59999]  =  1.66177e-32
  [9 , 59999]  =  3.24064e-40
  [10, 59999]  =  4.15921e-22
  [1 , 60000]  =  1.98666e-24
  [2 , 60000]  =  1.0
  [3 , 60000]  =  4.8494e-15
  [4 , 60000]  =  5.53727e-19
  [5 , 60000]  =  1.76141e-26
  [6 , 60000]  =  3.02188e-36
  [7 , 60000]  =  1.05937e-28
  [8 , 60000]  =  2.18159e-23
  [9 , 60000]  =  9.03462e-25
  [10, 60000]  =  8.88769e-14

In [77]:
grads = 
gradient(model) do model
    crossentropy(Array(softmax(model(X[:, 1:2]))), y[:, 1:2])
end[1];

In [22]:
model[1].W

100×784 SparseMatrixCSC{Float64,Int64} with 72328 stored entries:
  [14,   1]  =  1.0
  [21,   1]  =  1.0
  [22,   1]  =  1.0
  [38,   1]  =  1.0
  [54,   1]  =  1.0
  [69,   1]  =  2.0
  [70,   1]  =  1.0
  [89,   1]  =  1.0
  [10,   2]  =  1.0
  [22,   2]  =  2.0
  [27,   2]  =  1.0
  [33,   2]  =  1.0
  ⋮
  [93, 782]  =  1.0
  [96, 782]  =  1.0
  [2 , 783]  =  1.0
  [25, 783]  =  1.0
  [63, 783]  =  1.0
  [64, 783]  =  1.0
  [69, 783]  =  1.0
  [82, 783]  =  1.0
  [41, 784]  =  1.0
  [44, 784]  =  1.0
  [52, 784]  =  1.0
  [60, 784]  =  1.0
  [91, 784]  =  1.0

In [78]:
update_!(opt, model, grads)

Chain(SprAffine(784, 784, 0.377653), SprAffine(100, 100, 1.0), softmax)